# Stock Data Analysis

lets see where the data takes us ey ;)

In [19]:
import warnings

from datetime import datetime

import pandas as pd
import pandas_datareader as pdr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
warnings.filterwarnings('ignore')

guess we'll start with the standard suite of EDA checks - look at the head, look at the dtypes, look at the NULLS, etc...

In [17]:
df = pd.read_csv('data/sp500_measures_joe.csv')

df['Date'] = pd.to_datetime(df['Date'].str[:10], format='%Y-%m-%d')
df['Ticker'] = df['Ticker'].str.upper()

df.head()

,Date,Unix,Ticker,Price,stock_p_change,SP500,SP500_p_change,Market Cap,Enterprise Value,Trailing PE,...,200-Day Moving Average,Avg Vol (3 month),Shares Outstanding,Float,% Held by Insiders,% Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior month)
0,2007-07-11,1.184105e+09,CTAS,33.538876,-33.02,122.084023,-15.90,6.350000e+09,7.080000e+09,19.13,...,39.30,1064470.0,158650000.0,135580000.0,17.81,64.70,4970000.0,4.200,3.70,4490000.0
1,2006-02-28,1.141130e+09,CTAS,34.026257,-0.79,100.297188,11.94,6.910000e+09,7.160000e+09,22.55,...,41.44,940276.0,168020000.0,140190000.0,16.59,66.20,3460000.0,3.400,2.50,4600000.0
2,2005-04-20,1.113983e+09,CTAS,31.885536,9.65,87.811470,17.27,6.750000e+09,7.020000e+09,23.35,...,43.13,1027318.0,172090000.0,122500000.0,28.82,65.87,6170000.0,6.479,5.04,6360000.0
3,2005-02-22,1.109049e+09,CTAS,35.317966,-4.10,91.157219,10.92,7.550000e+09,7.670000e+09,26.61,...,43.75,994136.0,171720000.0,122200000.0,28.84,63.20,5420000.0,4.740,4.44,4640000.0
4,2013-04-26,1.366969e+09,CTAS,41.474506,30.78,143.779022,20.10,5.480000e+09,6.550000e+09,18.46,...,42.70,580981.0,122520000.0,102370000.0,16.36,74.60,5060000.0,8.000,5.60,5860000.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8682 entries, 0 to 8681
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        8682 non-null   datetime64[ns]
 1   Unix                        8682 non-null   float64       
 2   Ticker                      8682 non-null   object        
 3   Price                       8578 non-null   float64       
 4   stock_p_change              8578 non-null   float64       
 5   SP500                       8682 non-null   float64       
 6   SP500_p_change              8682 non-null   float64       
 7   Market Cap                  8593 non-null   float64       
 8   Enterprise Value            8548 non-null   float64       
 9   Trailing PE                 7887 non-null   float64       
 10  Forward PE                  8350 non-null   float64       
 11  PEG Ratio                   8215 non-null   float64     

ok I'd like to get a feel for the periodicity of the data actually, and then we can look to maybe build some extra rolling average columns if the data allows - I dont know in stock terms but for crypto the 20 week moving average usually acts as meaningful support so If the data is daily / weekly I'd like to slap that on there too

In [18]:
df.sort_values(['Ticker', 'Date'])

,Date,Unix,Ticker,Price,stock_p_change,SP500,SP500_p_change,Market Cap,Enterprise Value,Trailing PE,...,200-Day Moving Average,Avg Vol (3 month),Shares Outstanding,Float,% Held by Insiders,% Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior month)
4287,2004-01-30,1.075460e+09,A,23.393015,-41.37,85.583038,5.46,1.745000e+10,1.652000e+10,NaN,...,23.24,2989323.0,476150000.0,428500000.0,10.01,63.03,9300000.0,3.923,2.17,6930000.0
4288,2004-04-13,1.081800e+09,A,19.680344,-32.89,85.678711,5.64,1.532000e+10,1.477000e+10,NaN,...,27.32,3536636.0,480750000.0,432700000.0,10.00,66.79,7520000.0,2.328,1.74,9170000.0
4282,2004-12-13,1.102945e+09,A,15.129939,50.92,92.082664,7.57,1.161000e+10,1.037000e+10,33.58,...,25.64,2780227.0,487000000.0,389600000.0,20.00,66.68,6930000.0,2.638,1.78,6540000.0
4279,2005-01-22,1.106342e+09,A,13.835259,57.11,89.758362,9.77,1.071000e+10,9.760000e+09,30.70,...,24.72,2459863.0,491070000.0,441400000.0,10.12,67.08,7040000.0,2.385,1.60,6930000.0
4286,2006-01-09,1.136745e+09,A,21.926983,4.47,100.719551,11.58,1.712000e+10,1.487000e+10,52.82,...,30.86,3843670.0,494180000.0,NaN,0.57,70.40,10980000.0,2.300,2.20,5840000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2012-11-01,1.351759e+09,ZNGA,2.230000,60.54,128.292755,26.07,1.730000e+09,5.330600e+08,NaN,...,4.81,18107200.0,783980000.0,545800000.0,8.56,38.60,19330000.0,1.000,12.20,18320000.0
216,2012-11-10,1.352479e+09,ZNGA,2.120000,63.21,124.098061,31.13,1.660000e+09,4.703500e+08,NaN,...,4.51,17387300.0,783980000.0,545390000.0,8.56,38.60,19330000.0,1.000,12.20,18320000.0
213,2013-02-16,1.360948e+09,ZNGA,3.200000,52.19,137.597244,23.42,2.470000e+09,1.320000e+09,NaN,...,2.65,26601900.0,771530000.0,535490000.0,8.56,38.60,13150000.0,0.600,8.30,14750000.0
221,2013-04-22,1.366568e+09,ZNGA,3.230000,41.18,141.898209,22.74,2.510000e+09,1.320000e+09,NaN,...,2.79,36017100.0,785490000.0,472190000.0,5.81,30.20,27660000.0,0.900,17.50,28210000.0


... this is pretty squiff lol. Prehaps it would be worth using pandas_datareader to pull the stock price down for the missing dates and just interpolating the other colunmns between the known dates? - better yet keep the data all relational and the prices separate from the rest could be a shout - assuming this sp_500 data does in fact have 500 tickers there could be quite alot of data involved in the download. (i.e. 10 years * 365 days * ~500 tickers = 1,825,000)

In [20]:
tickers = df['Ticker'].unique().tolist()
start_date = df['Date'].min().strftime('%Y-%m-%d')
end_date = df['Date'].max().strftime('%Y-%m-%d')

prices = pdr.DataReader(tickers, data_source='yahoo', start=start_date, end=end_date)
prices.to_csv('data/raw_price_output.csv')
prices.head()

Attributes  Adj Close                                                       \
Symbols          CTAS         VZ       GME       AMZN       CNP        CAT   
Date                                                                         
2003-10-02  29.806219  12.656958  5.581658  50.090000  4.281927  22.713688   
2003-10-03  31.261112  12.831902  5.615324  52.889999  4.309852  22.926140   
2003-10-06  31.300026  12.854723  5.807212  54.150002  4.333123  23.122976   
2003-10-07  31.797960  12.740627  5.807212  54.910000  4.323814  23.251074   
2003-10-08  31.821306  12.573266  5.749982  55.700001  4.300541  23.529135   

Attributes                               ... Volume                            \
Symbols         AAPL      BF-B ZNGA  PM  ...    STI DSW TMK UTX LUK FLIR PCLN   
Date                                     ...                                    
2003-10-02  0.314928  5.976847  NaN NaN  ...    NaN NaN NaN NaN NaN  NaN  NaN   
2003-10-03  0.332075  5.983544  NaN NaN  ...    NaN NaN NaN NaN NaN  NaN  NaN   
2003-10-06  0.341262  6.060903  NaN NaN  ...    NaN NaN NaN NaN NaN  NaN  NaN   
2003-10-07  0.355500  6.177687  NaN NaN  ...    NaN NaN NaN NaN NaN  NaN  NaN   
2003-10-08  0.353051  6.125622  NaN NaN  ...    NaN NaN NaN NaN NaN  NaN  NaN   

Attributes               
Symbols    DLPH NYX  XL  
Date                     
2003-10-02  NaN NaN NaN  
2003-10-03  NaN NaN NaN  
2003-10-06  NaN NaN NaN  
2003-10-07  NaN NaN NaN  
2003-10-08  NaN NaN NaN  

[5 rows x 2826 columns]

yikes that took its time to download - Im thinking save to a parquet per year and then swap to pyspark for the continuation fo what we doin' here bois as I think using this in local pandas may be a bit of a stretch. gives a chance to go back o some big data tools at least and prehaps even to get a chance to use spark.ml. might even gzip it for old times sake

In [81]:
prices = pd.read_csv('data/raw_price_output.csv', index_col=[0], header=[0,1])  # so dont have to re-download

prices = prices.reset_index().melt(id_vars='Date')
prices = prices.pivot(index=['Date', 'Symbols'], columns='Attributes', values='value').reset_index()
prices.dropna(axis=0, inplace=True)

prices['Date'] = pd.to_datetime(prices['Date'], format='%Y-%m-%d')
prices['year'] = prices['Date'].dt.year
prices.columns = [col.lower().replace(' ', '_') for col in prices.columns]

prices.to_parquet('data/prices_pivoted', compression='gzip', index=False, partition_cols=['year'])

prices.head()

,date,symbols,adj_close,close,high,low,open,volume,year
0,2003-10-02,A,13.544954,15.729614,15.865522,15.622318,15.808298,3370159.0,2003
1,2003-10-02,AA,51.975750,65.193390,65.842201,64.280251,64.592644,1570787.0,2003
2,2003-10-02,AAPL,0.314928,0.367321,0.371429,0.362143,0.371429,204058400.0,2003
4,2003-10-02,ABC,11.019091,14.077500,14.140000,13.812500,13.812500,5980800.0,2003
5,2003-10-02,ABT,12.501964,19.370073,19.428431,19.145622,19.208469,9901740.0,2003


beautiful, although probs shouldve renamed 'symbols' to 'ticker'. whoops. I'll just check the coverage as I'm fairly sure that dropna's command lost me at least a couple of tickers entirely?

In [86]:
syms = prices['symbols'].unique().tolist()
tickers = df['Ticker'].unique().tolist()

missing_ticks = [ticker for ticker in tickers if ticker not in syms]

print(missing_ticks)
print(f'\nthe percentage of tickers missing from prices df is {round(len(missing_ticks) / len(tickers) * 100)}%')

['AGN', 'CBG', 'WPX', 'HRS', 'PX', 'CELG', 'TIF', 'CBS', 'RTN', 'JCP', 'ALXN', 'DF', 'APC', 'ETFC', 'ESV', 'DO', 'HCN', 'IR', 'LM', 'WIN', 'SHLD', 'RSH', 'SFLY', 'DNB', 'SCG', 'COG', 'NTRI', 'CCE', 'VAR', 'WTW', 'RDC', 'S', 'DPS', 'MON', 'RHT', 'WYN', 'FTR', 'CHK', 'SNE', 'SKS', 'JEC', 'HCP', 'MYL', 'NE', 'AVP', 'SAI', 'SYMC', 'MOLX', 'NBL', 'VIAB', 'TSS', 'BKS', 'DNR', 'QEP', 'BBT', 'CTL', 'GCI', 'STI', 'DSW', 'TMK', 'UTX', 'LUK', 'FLIR', 'PCLN', 'DLPH', 'NYX', 'XL']

the percentage of tickers missing from prices df is 14%


cock - we've lost 14% of the tickers. I guess this cant be avoided though so ack well moving swiftly on